In [2]:
!pip install kaggle

In [4]:
!mkdir .kaggle

���� ���͸� �Ǵ� ���� .kaggle��(��) �̹� �ֽ��ϴ�.


In [6]:
!pip install opendatasets

In [8]:
import opendatasets as od

In [10]:
od.download("https://www.kaggle.com/competitions/lyft-motion-prediction-autonomous-vehicles/rules")

Skipping, found downloaded files in ".\lyft-motion-prediction-autonomous-vehicles" (use force=True to force download)


In [12]:
import os

path="./lyft-motion-prediction-autonomous-vehicles"
files=os.listdir(path)
print(files)


#'aerial_map'==위성 지도 데이터(도로, 차선 정보) 
#'meta.json'==데이터셋의 메타정보(버전, 데이터 통계)
#'multi_mode_sample_submission.csv'==다중 모드 예측 샘플 제출 파일, 차량의 여러가지 이동 경로 예측
#'scenes'==개별 주행 장면 데이터  센서 정보, 차량 궤적 포함
#'semantic_map'==의미론적 지도 데이터  도로 유형, 교차로 정보, 차선 정보
#'single_mode_sample_submission.csv'==단일 모드 예측 샘플 제출 파일    차량의 단일 이동 경로 예측


['aerial_map', 'meta.json', 'multi_mode_sample_submission.csv', 'scenes', 'semantic_map', 'single_mode_sample_submission.csv']


In [19]:
import json

with open("./lyft-motion-prediction-autonomous-vehicles/meta.json", "r") as f:
    meta = json.load(f)

print(json.dumps(meta, indent=4))



"""world_to_ecef=세계 좌표->ECEF(Earth-Centered, Earth-Fixed) 4*4 변환 행렬
                            위도, 경도, 고도를 지구 중심 좌표계로 변환
                            자율주행 차량의 GPS 데이터와 정밀지도 데이터를 일치시킴
    ecef_to_aerail=ECEF->항공지도 변환 행렬
                            aerail_map/(위성지도) 사용시 적용하는 변환행렬
                            ECEF좌표를 픽셀 좌표로 변환
                            semantic_map/(의미론적 지도)와의 정합에 필요
"""

{
    "world_to_ecef": [
        [
            0.846617444,
            0.323463078,
            -0.422623402,
            -2698767.44
        ],
        [
            -0.532201938,
            0.514559352,
            -0.672301845,
            -4293151.58
        ],
        [
            -3.05311332e-16,
            0.794103464,
            0.6077826,
            3855164.76
        ],
        [
            0.0,
            0.0,
            0.0,
            1.0
        ]
    ],
    "ecef_to_aerial": [
        [
            -0.717416495,
            -1.14606296,
            -1.62854453,
            -572869.824
        ],
        [
            1.80065798,
            -1.08914046,
            -0.0287877303,
            300171.963
        ],
        [
            0.0,
            0.0,
            0.0,
            0.0
        ],
        [
            0.0,
            0.0,
            0.0,
            1.0
        ]
    ]
}


'world_to_ecef=세계 좌표->ECEF(Earth-Centered, Earth-Fixed) 4*4 변환 행렬\n                            위도, 경도, 고도를 지구 중심 좌표계로 변환\n                            자율주행 차량의 GPS 데이터와 정밀지도 데이터를 일치시킴\n    ecef_to_aerail=ECEF->항공지도 변환 행렬\n                            aerail_map/(위성지도) 사용시 적용하는 변환행렬\n                            ECEF좌표를 픽셀 좌표로 변환\n                            semantic_map/(의미론적 지도)와의 정합에 필요\n'

In [22]:
os.listdir('./lyft-motion-prediction-autonomous-vehicles/scenes')

#'mask.npz'==numpy 압축 배열 파일  특정 data filtering 사용(ex: 가려진 영역, 선택된 샘플)
#'sample.zarr'==sample dataset(테스트용 작은 데이터)   .zarr형식은 대용량 데이터 다룰 때 사용, 압축된 배열 저장소
#'train.zarr'==훈련 데이터   자율주행 차량의 주행 궤적과 센서 데이터 포함
#'validate.zarr'==검증 데이터    훈련된 모델이 잘 작동하는지 평가
#'test.zarr'==테스트 데이터     실제 대회에서 평가되는 데이터

['mask.npz', 'sample.zarr', 'test.zarr', 'train.zarr', 'validate.zarr']

In [ ]:
    """
    목표: 자율주행 차량의 주행경로 예측을 위한 모델 개발.
          주어진 데이터를 활용하여 주변 객체들의 움직임을 예측하는 모델 구현
          자율주행 차량이 주변의 차량, 자전거, 보행자 등의 움직임을 정확하게 예측하여 안전하고 효율적인 주행을 할 수 있도록 할 것
    데이터: Lyft의 자율주행 차량이 수집한 1,000시간 이상의 주행 데이터로 구성
            각 데이터는 에피소드, 시간별 차량 위치, 센서로 감지된 객체 정보 등을 포함
            
    사전조사 및 준비사항
    1. 자율주행 기술 및 모션 예측
        a. 자율주행 차량의 기본 원리, 모션 예측의 중요성에 대한 이해 필요
        b. 모션 예측은 자율주행 차량이 주변 객체들의 미래 움직임을 예측해 안전한 주행 경로 계획
    
    2. 데이터 분석 및 전처리
        a. 제공된 데이터셋의 구조와 특징 파악
        b. 데이터는 주행 에피소드, 시간별 프레임, 감지된 객체 등의 정보를 포함하여 각 객체의 속도, 위치, 방향 등의 속성 분석
    
    3. 모델 개발 및 평가:
        a. 과거 데이터 기반으로 객체들의 미래 위치 예측하는 모델 설계
        b. 평가 지표로는 예측의 정확도, 정밀도 등 사용
        
    4. 필요한 기술 스택, 도구
        a. python(머신러닝과 데이터 분석)
        b. c++(자율주행 라이브러리와 같은 고성능 연산이 필요한 경우 사용 가능)
        c. NumPy(다차원 배열 및 행렬 연산)
        d. Pandas(데이터 분석 및 전처리)
        e. Matplotlib,Seaborn(데이터 시각화)
        f. Shapely,GeoPandas(좌표 및 경로 데이터 처리)-지리정보처리
        
            i) Lyft 데이터는 주행 궤적 정보가 포함되어있어 지리정보처리 라이브러리가 유용함
        
        g. PyTorch(딥러닝 프레임워크)
        h. TensorFlow,Keras(pytorch대안)
        i. Scikit-learn(기초적 머신러닝 모델 및 전처리)
        j. XGBoost,LightGBM(트리 기반 모델, 베이스 라인으로 활용 가능)
        
            i) 모션 예측 문제는 시계열 예측, 딥러닝 모델 활용하기 때문에 pytorch, tensorflow 활용 많이 함
            ii) 트리 기반은 비교용으로 사용 많이 함
        
        k. LSTM,GRU(시계열 데이터 예측)
        l. Transformers(최근 모션 예측에 많이 사용)
        m. 1D CNN(시계열 데이터 특징 추출)
        o. Kalman Filter(이동 궤적 예측 및 필터링)
        
            i) LSTM,GRU는 RNN기반 모델
            ii) Kalman Filter은 자율주행에서 센서 데이터 보정시 사용
            
        p. CARLA(자율주행 시뮬레이터)
        q. OpenCV(자율주행 센서 데이터 활용 가능)
        r. Pytorch3D(3D데이터 처리)
        s. Waymo Open Dataset(자율주행 관련 데이터셋 활용)
        
            i) Lyft 데이터의 핵심은 차량의 궤적 예측
            ii) 차량 움직임 예측시 3D데이터 처리가 필요
        
        t. HDF5,Parquet(대용량 데이터 저장)
        u. SQLite,PostgreSQL(데이터베이스 저장)
        v. Dask(대규모 데이터 병렬 처리)
        
    """